<a href="https://colab.research.google.com/github/Janamawasy/ML---TreeANN/blob/main/tree_ANN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
#importing the dataset
import numpy as np
import pandas as pd
import tensorflow as tf
from datetime import datetime
tf.__version__



'2.15.0'

In [4]:
# import data
dataset = pd.read_csv('Tree_Data.csv')

# -------------------- DATA PREPROCESSING --------------------------------

# data processing :: drop Adult column
dataset = dataset.drop(dataset.columns[8], axis=1)

dataset['PlantDate'] = dataset['PlantDate'].apply(lambda x: int(datetime.strptime(x, '%m/%d/%y').timestamp()))
dataset = dataset.dropna(subset=['Event'])


df = dataset.iloc[:, 1:-3]
# y is Event
y = dataset.iloc[:, -3]
# y is Alive
# y = dataset.iloc[:, -1]
# category_mapping = {'X': 1, np.nan: 0}
# y = y.map(category_mapping)


# Fill NaN vals in'EMF'
column_mean = df['EMF'].mean()
df['EMF'] = df['EMF'].fillna(column_mean)


nan_count_x = df.isna().sum()
nan_count_y = np.isnan(y).sum()
# print(nan_count_x, nan_count_y)

In [7]:
# -------------- ENCODING NON NUMERIC DATA ------------------------------------
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.compose import ColumnTransformer

# LABEL ENCODING
# features that have only 2 categories, or have order logic => Label Encoder
label_enc_fea = ['Sterile','Myco','Light_Cat']
# Define a mapping for categories
category_mapping = {'High': 2, 'Med': 1, 'Low': 0}

# Use the map function to replace categories with numerical values
df['Light_Cat'] = df['Light_Cat'].map(category_mapping)

le = LabelEncoder()
df['Sterile'] = le.fit_transform(df['Sterile'])
df['Myco'] = le.fit_transform(df['Myco'])

# ONE HOT ENCODING
OH_enc_feat = ['Subplot','Species','Soil','Conspecific','SoilMyco']

ct = ColumnTransformer(transformers=[('encoder', OneHotEncoder(), OH_enc_feat)], remainder='passthrough')
df = ct.fit_transform(df)


In [10]:
# -------------------- CORR +  DELETING CORRELATED FEATURES

# Set the correlation threshold
correlation_threshold = 0.7

# Calculate the correlation matrix
correlation_matrix = pd.DataFrame(df).corr()

# Find features with correlation above the threshold
highly_correlated_features = set()
for i in range(len(correlation_matrix.columns)):
    for j in range(i):
        if abs(correlation_matrix.iloc[i, j]) >= correlation_threshold:
            colname = correlation_matrix.columns[i]
            highly_correlated_features.add(colname)

print(np.array(highly_correlated_features))

indices_to_drop = [pd.DataFrame(df).columns.get_loc(col) for col in highly_correlated_features]
filtered_data = np.delete(df, indices_to_drop, axis=1)
correlation_matrix = pd.DataFrame(filtered_data).corr()
print((abs(correlation_matrix) >= 0.7).sum().sum()-len(correlation_matrix) == 0)



{32, 33, 35, 18, 20, 21, 24, 26, 31}
True


In [193]:
# ------------ CORR BETWEEN X AND Y
# correlation_with_y = df.corrwith(np.array(y))

x_transposed = np.transpose(filtered_data)

# Compute the correlation between each feature and y
correlation_with_y = np.corrcoef(x_transposed, y)[0:-1, -1]
print(correlation_with_y)


[ 0.00712179 -0.00445703 -0.00449648 -0.00302205  0.01037145 -0.27938572
 -0.23848294  0.30706699  0.23777162  0.02658269 -0.01365947 -0.00273528
  0.01619982 -0.03631977  0.0081799   0.00136805 -0.04125583  0.03033442
  0.02036334 -0.01436951  0.04354648  0.06603677  0.461038   -0.27244639
 -0.08373511  0.02273629  0.89574222]


In [11]:
# ------------------------------ANN MODEL------------------------------------
from sklearn.model_selection import train_test_split

x = np.array(filtered_data)
y = np.array(y)

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=0)

# scaling data - normilizing
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
x_train = sc.fit_transform(x_train)
x_test = sc.fit_transform(x_test)

#Training

#Init the ANN
ann = tf.keras.models.Sequential()

ann.add(tf.keras.layers.Dense(units=6, activation='relu'))
ann.add(tf.keras.layers.Dense(units=6, activation='relu'))
ann.add(tf.keras.layers.Dense(units=6, activation='relu'))
ann.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))

ann.compile(optimizer='adam', loss='binary_crossentropy', metrics= ['accuracy'])

# Actual training
ann.fit(x_train, y_train, batch_size=32, epochs=100)

y_pred = ann.predict(x_test)


Epoch 1/100
70/70 [==============================] - 2s 3ms/step - loss: 0.6743 - accuracy: 0.5335
Epoch 2/100
70/70 [==============================] - 0s 1ms/step - loss: 0.6275 - accuracy: 0.5762
Epoch 3/100
70/70 [==============================] - 0s 2ms/step - loss: 0.5909 - accuracy: 0.6894
Epoch 4/100
70/70 [==============================] - 0s 2ms/step - loss: 0.5602 - accuracy: 0.7276
Epoch 5/100
70/70 [==============================] - 0s 2ms/step - loss: 0.5276 - accuracy: 0.7771
Epoch 6/100
70/70 [==============================] - 0s 2ms/step - loss: 0.4997 - accuracy: 0.7951
Epoch 7/100
70/70 [==============================] - 0s 1ms/step - loss: 0.4812 - accuracy: 0.8018
Epoch 8/100
70/70 [==============================] - 0s 1ms/step - loss: 0.4679 - accuracy: 0.8045
Epoch 9/100
70/70 [==============================] - 0s 1ms/step - loss: 0.4580 - accuracy: 0.8076
Epoch 10/100
70/70 [==============================] - 0s 1ms/step - loss: 0.4498 - accuracy: 0.8094
Epoch 11/

In [12]:
from sklearn.metrics import multilabel_confusion_matrix,accuracy_score, confusion_matrix

y_binary_pred = np.where(y_pred > 0.5, 1, 0)

cm = confusion_matrix(y_test,y_binary_pred)
acc = accuracy_score(y_test,y_binary_pred)


In [13]:
print(cm)
print('acc is:', acc)

[[235  13]
 [ 21 288]]
acc is: 0.9389587073608617
